In [42]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [43]:
data = pd.read_parquet("../data/tables/tbl_merchants.parquet")

# Remove revenue level and take rate
data['tags']= data['tags'].str[0:-25]

In [44]:
STOPWORDS = set(stopwords.words('english'))

# Function for preprocessing tags text
def preprocess_text(x):
    
    # Lower case and remove all punctutations
    x = x.lower()
    x = x.translate(str.maketrans('', '', string.punctuation))

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    x = " ".join(lemmatizer.lemmatize(word) for word in x.split())

    # Remove stop words
    x =  ' '.join([word for word in x.split() if word not in STOPWORDS])

    return x

In [45]:
data['tags_clean'] = data['tags'].map(preprocess_text)

In [46]:
# One Hot Encode tags
count_vectorizer = CountVectorizer(binary=True)
datavec  = count_vectorizer.fit_transform(data['tags_clean'])
count_array = datavec.toarray()
df = pd.DataFrame(data=count_array,columns = count_vectorizer.get_feature_names())

# sns.pairplot(df)
name = count_vectorizer.get_feature_names()


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
# list of tags
km = KMeans(n_clusters=3, random_state=0)
df_clust = km.fit_transform(df)
clusters = km.fit_predict(df)

In [52]:
data['clusters'] = km.labels_
# print(data.where(data['clusters'] == 2))
data

,name,tags,tags_clean,clusters,merchant_segments
merchant_abn,,,,,
10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...",furniture home furnishing equipment shop manuf...,2,Furnitures
10142254217,Arcu Ac Orci Corporation,"([cable, satellite, and otHer pay television a...",cable satellite pay television radio service,1,Appliances
10165489824,Nunc Sed Company,"([jewelry, watch, clock, and silverware shops]",jewelry watch clock silverware shop,0,Accessories
10187291046,Ultricies Dignissim Lacus Foundation,"([wAtch, clock, and jewelry repair shops]",watch clock jewelry repair shop,0,Accessories
10192359162,Enim Condimentum PC,"([music shops - musical instruments, pianos, a...",music shop musical instrument piano sheet music,0,Accessories
...,...,...,...,...,...
99938978285,Elit Dictum Eu Ltd,"[(opticians, optical goods, and eyeglasses)",optician optical good eyeglass,1,Appliances
99974311662,Mollis LLP,"((books, periodicals, and newspapers)",book periodical newspaper,1,Appliances
99976658299,Sociosqu Corp.,((shoe shops),shoe shop,0,Accessories


In [51]:
cluster_mapping = {0: 'Accessories', 1: 'Appliances', 2: 'Furnitures'}
data['merchant_segments'] = data['clusters'].map(cluster_mapping)

